In [3]:
# %% [markdown]
# ## 高频加密货币交易策略（修复版）
# 修复内容：
# 1. 列名格式标准化
# 2. 时间序列排序
# 3. 数据类型转换

# %%
import pandas as pd
import numpy as np
import ccxt
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy

# %% [markdown]
# ### 数据获取与格式处理

# %%
def fetch_ohlcv(symbol='BTC/USDT', timeframe='1m', limit=1000):
    exchange = ccxt.binance({
        'httpsProxy': 'http://127.0.0.1:7890',  # 设置代理
        'enableRateLimit': True,
        'options': {
            'adjustForTimeDifference': True
        }
    })
    data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

    # 转换列名为首字母大写格式
    df = pd.DataFrame(data, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

    # 时间处理
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df.set_index('Timestamp', inplace=True)
    df.sort_index(inplace=True)  # 确保时间升序排列

    # 类型转换
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

# %%
# 获取并处理数据
df = fetch_ohlcv(limit=5000)  # 获取最近5000根1分钟K线
df = df.iloc[-3000:]  # 取最近3000分钟数据

# 验证列名格式
print("数据列名验证:", df.columns.tolist())
df.head()

# %% [markdown]
# ### 修复后的策略类

# %%
class OptimizedHFTStrategy(Strategy):
    # 优化策略参数
    fast_ma = 6
    slow_ma = 18
    atr_period = 14
    risk_per_trade = 0.015  # 调整风险系数

    def init(self):
        # 使用标准化列名
        close = self.data.Close
        self.fast_ma = self.I(lambda x: x.rolling(self.fast_ma).mean(), close)
        self.slow_ma = self.I(lambda x: x.rolling(self.slow_ma).mean(), close)
        self.atr = self.I(lambda x: x.rolling(self.atr_period).mean(),
                         self.data.High - self.data.Low)

    def next(self):
        price = self.data.Close[-1]
        atr = self.atr[-1]

        # 添加交易过滤条件
        if not self.position and atr > 0:
            position_size = (self.risk_per_trade * self.equity) / atr

            # 多空信号逻辑
            if self.fast_ma[-1] > self.slow_ma[-1]:
                self.buy(size=position_size,
                        sl=max(price - 2*atr, price*0.995),  # 防极端值
                        tp=price + 4*atr)

            elif self.fast_ma[-1] < self.slow_ma[-1]:
                self.sell(size=position_size,
                         sl=min(price + 2*atr, price*1.005),
                         tp=price - 4*atr)

# %% [markdown]
# ### 执行回测

# %%
# 初始化回测引擎
bt = Backtest(
    df,
    OptimizedHFTStrategy,
    cash=10000,
    commission=0.00075,  # Binance现货手续费
    exclusive_orders=True,
    trade_on_close=True   # 使用收盘价成交
)

# 运行回测
stats = bt.run()
print(stats)

# %% [markdown]
# ### 增强分析功能

# %%
# 绘制优化后的资金曲线
bt.plot(resample=False, plot_volume=False)

# 生成详细绩效报告
from backtesting.lib import plot_heatmaps

plt.figure(figsize=(12,6))
plt.title('动态回撤分析')
plot_heatmaps(bt.data.returns, bt.data.equity)
plt.show()

# 显示关键指标
print(f"""
策略绩效报告：
总收益率: {stats['Return [%]']:.2f}%
年化收益率: {stats['Return (Ann.) [%]']:.2f}%
最大回撤: {stats['Max. Drawdown [%]']:.2f}%
夏普比率: {stats['Sharpe Ratio']:.2f}
交易次数: {stats['# Trades']}
胜率: {stats['Win Rate [%]']:.2f}%
平均盈亏比: {stats['Avg. Trade Ratio']:.2f}
""")

数据列名验证: ['Open', 'High', 'Low', 'Close', 'Volume']


C:\Users\youlong\AppData\Local\Temp\ipykernel_21040\4116106933.py:93: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


RuntimeError: Indicator "λ(C)" error. See traceback above.